In [1]:
import sys
import os
myDir = os.getcwd()
parentDir = os.path.abspath(os.path.join(myDir, os.pardir))
sys.path.append(parentDir)

from pathlib import Path
path = Path(parentDir)
a=str(path.parent.absolute())

sys.path.append(a)

In [2]:
from MetaX.utils.taxaFuncAnalyzer import TaxaFuncAnalyzer
from MetaX.utils.taxaFuncPloter.basic_plot import BasicPlot


In [3]:
df_path = './example_data/SW_TaxaFunc.tsv'
meta_path = './example_data/SW_Meta.tsv'
sw = TaxaFuncAnalyzer(df_path, meta_path)

original df shape: (3999, 229)
after remove all zero row: (3999, 229)


In [4]:
sw.set_func('Description')
sw.set_multi_tables(level='s')

sw.set_group('Person')

batch_list is not set, Batch effect removal did not perform.
transform_method is not set, data transform did not perform.
normalize_method is not set, data normalization did not perform.
Original data shape: (3999, 229)
Starting to set Function table...
Function number: 830
Starting to set Taxa table...
Taxa number: 340
Taxa-Function number: 1272
group is set to Person
 {'V5', 'V2', 'V4', 'V1', 'V3'}


In [5]:
sw.clean_df.shape

(1557, 193)

In [6]:
sw.peptide_df

,V1_01,V1_02,V1_03,V1_04,V1_05,V1_06,V1_07,V1_08,V1_09,V1_10,...,V5_29,V5_30,V5_31,V5_32,V5_33,V5_34,V5_35,V5_36,V5_37,V5_38
Sequence,,,,,,,,,,,,,,,,,,,,,
(AccQTag)QPVENAVVLGVVAELAK,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
(Acetyl)ADADIASDDAIKK,0,867943,0,0,0,0,0,0,0,0,...,0,0,0,1016152,1033304,0,253807,0,558854,0
(Acetyl)AEKNVEVIDKESK,0,0,0,459568,394789,323510,0,0,0,430644,...,0,0,866959,0,907214,0,0,0,0,1044975
(Acetyl)AEVEAVKTEE(Hydroxamic_acid)PAK,0,0,0,0,0,1063171,0,0,0,0,...,2023303,0,2492253,970270,11312311,1007825,760036,826901,887036,3858052
(Acetyl)APAAAPAPAPAPAAAPAAAGGQTVEAPMPGK,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
AAVDAGWIP(Pro->Ala)HAHQVGQTGK,0,0,0,0,0,0,0,0,0,0,...,0,0,645934,807673,414891,865358,0,0,0,1049619
AAVDAGWIPHAHQVGQTGK,569965,114438,885880,150124,148181,124527,318044,0,180868,636715,...,1202395,1998095,1983834,614243,2193489,1585702,1384530,2306746,3150484,2688650
AAVDALFNTLR,291204,0,0,0,0,0,118773,413122,251647,0,...,1855737,2706241,1164897,1588759,1211888,0,1784060,1181814,1858872,1570889


In [8]:
index_list = sw.taxa_func_df.index.tolist()
index_list = [f"{i[0]} <{i[1]}>" for i in index_list]
print(type(index_list))

<class 'list'>
